In [1]:
# Initialization

%matplotlib notebook

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
from IPython.core.display import HTML

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.graph_objects as go

import sklearn.metrics
import scipy.integrate as integrate
import scipy.optimize as sco

import fluids as fds
import ht 

import general as gen
import model as ty
import proc as pr
import heat_transfer as bht
import plot_functions_here as pfun
# importing sys
import sys


import scipy.constants as scc

In [ ]:
T_s = 315
T_amb = 291
speed = 3.
longueur = 1.7

T_mean = (T_s+T_amb)/2

g = scc.g

rho = bht.air_rho(T_mean)
Cp = bht.air_c_p()
mu = bht.air_mu(T_mean)
nu = bht.air_nu(T_mean)
lambd = bht.air_k(T_mean)
alpha = (lambd)/(rho*Cp)
Pr = bht.air_Pr()
beta = 1/T_mean

Re_c = 3.5 * 10**5

x_c = (nu*Re_c)/speed

In [ ]:
x_c

In [ ]:
if x_c < longueur:
    lam = integrate.quad(bht.Nu_forced_flat_plate_isoflux_lam,0,x_c,args=(lambd,speed,nu,Pr))[0]
    turb = integrate.quad(bht.Nu_forced_flat_plate_isoflux_turb,x_c,longueur,args=(lambd,speed,nu,Pr))[0]
else:
    lam = integrate.quad(bht.Nu_forced_flat_plate_isoflux_lam,0,longueur,args=(lambd,speed,nu,Pr))[0]
    turb = 0.

In [ ]:
(1/longueur)*(lam+turb)

In [ ]:
turbT = integrate.quad(bht.Nu_forced_flat_plate_isoflux_turb,0.,longueur,args=(lambd,speed,nu,Pr))[0]

In [ ]:
(1/longueur)*turbT

In [ ]:
dfu = pd.DataFrame(np.linspace(0.1,4.,10),columns=['u'])

In [ ]:
dfu['lam+turb'] = dfu.apply(lambda x : bht.h_top_forced(T_s,T_amb,x['u'],longueur),axis=1)

In [ ]:
dfu['turb'] = dfu.apply(lambda x : bht.h_top_forced_turbulent(T_s,T_amb,x['u'],longueur),axis=1)

In [ ]:
dfu['ratio'] = dfu['turb']/dfu['lam+turb']

In [ ]:
dfu

In [ ]:
plt.plot(dfu['u'],dfu['lam+turb'])
plt.plot(dfu['u'],dfu['turb'])
plt.show()

In [ ]:
# Set up the figure and axis
fig, ax = plt.subplots()

# Set the axis limits
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)

# Create the line
line, = ax.plot([], [], 'o-', lw=2)

# Initialize the dot's position and speed
x = [5]  # x-coordinate of the dot
y = [0]  # y-coordinate of the dot
u = 0.2  # speed of the dot (units per frame)

# Update function for the animation
def update(frame):
    # Update the dot's position
    y[0] += u

    # Update the line data
    line.set_data(x, y)

    return line,

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=100, interval=50, blit=True)

# Display the animation
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.animation import FuncAnimation

df = pd.read_csv('city_populations.csv',
				usecols=['name', 'group', 'year', 'value'])

colors = dict(zip(['India','Europe','Asia',
				'Latin America','Middle East',
				'North America','Africa'],
					['#adb0ff', '#ffb3ff', '#90d595',
					'#e48381', '#aafbff', '#f7bb5f',
					'#eafb50']))

group_lk = df.set_index('name')['group'].to_dict()

def draw_barchart(year):
	dff = df[df['year'].eq(year)].sort_values(by='value',
											ascending=True).tail(10)
	ax.clear()
	ax.barh(dff['name'], dff['value'],
			color=[colors[group_lk[x]] for x in dff['name']])
	dx = dff['value'].max() / 200
	
	for i, (value, name) in enumerate(zip(dff['value'],
										dff['name'])):
		ax.text(value-dx, i,	 name,		
				size=14, weight=600,
				ha='right', va='bottom')
		ax.text(value-dx, i-.25, group_lk[name],
				size=10, color='#444444',
				ha='right', va='baseline')
		ax.text(value+dx, i,	 f'{value:,.0f}',
				size=14, ha='left', va='center')
		
	# polished styles
	ax.text(1, 0.4, year, transform=ax.transAxes,
			color='#777777', size=46, ha='right',
			weight=800)
	ax.text(0, 1.06, 'Population (thousands)',
			transform=ax.transAxes, size=12,
			color='#777777')
	
	ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
	ax.xaxis.set_ticks_position('top')
	ax.tick_params(axis='x', colors='#777777', labelsize=12)
	ax.set_yticks([])
	ax.margins(0, 0.01)
	ax.grid(which='major', axis='x', linestyle='-')
	ax.set_axisbelow(True)
	ax.text(0, 1.12, 'The most populous cities in the world from 1500 to 2018',
			transform=ax.transAxes, size=24, weight=600, ha='left')
	
	ax.text(1, 0, 'by @pratapvardhan; credit @jburnmurdoch',
			transform=ax.transAxes, ha='right', color='#777777',
			bbox=dict(facecolor='white', alpha=0.8, edgecolor='white'))
	plt.box(False)
	plt.show()

fig, ax = plt.subplots(figsize=(15, 8))
animator = FuncAnimation(fig, draw_barchart,
						frames = range(1990, 2019))
plt.show()
